In [ ]:
# Trying Temporal Convolutional Network (TCN) using keras and tensorflow
import os
import random
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from tcn import TCN

# 0) Setting constant seeds for reproducability of results
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['CUDA_VISIBLE_DEVICES'] = ''

# 1) Setting configuration
FILE_PATH     = '../output/btc_final.csv'
DATE_COL      = 'date'
TARGET_COL    = 'Close'
WINDOW_SIZE   = 30
PRED_DAYS     = 30
EPOCHS        = 350
BATCH_SIZE    = 16
LEARNING_RATE = 1e-2

# 2) Loading data and preparation
df = pd.read_csv(FILE_PATH, parse_dates=[DATE_COL])
df.sort_values(DATE_COL, inplace=True)
df.set_index(DATE_COL, inplace=True)

# 2.1) Numeric features & target
data = df.select_dtypes(include=[np.number])
X_raw = data.values
y_raw = data[[TARGET_COL]].values

# 2.2) Scale
feature_scaler = MinMaxScaler()
X_scaled = feature_scaler.fit_transform(X_raw)

target_scaler = MinMaxScaler()
y_scaled = target_scaler.fit_transform(y_raw)

# 2.3) Window creation
def create_windows(X, y, window):
    Xs, ys = [], []
    for i in range(len(X) - window):
        if not (np.isnan(X[i:i+window]).any() or np.isnan(y[i+window])):
            Xs.append(X[i:i+window])
            ys.append(y[i+window])
    return np.array(Xs), np.array(ys)

X_win, y_win = create_windows(X_scaled, y_scaled, WINDOW_SIZE)

# 2.4) Split train/test by last PRED_DAYS windows
X_train, y_train = X_win[:-PRED_DAYS], y_win[:-PRED_DAYS]
X_test, y_test   = X_win[-PRED_DAYS:], y_win[-PRED_DAYS:]

# 2.5) Retrieve corresponding test dates
all_dates = df.index[WINDOW_SIZE:]
test_dates = all_dates[-PRED_DAYS:]

# 3) Building & Train Model
model = Sequential([
    TCN(nb_filters=64, kernel_size=2, dilations=[1,2,4], dropout_rate=0.2,
        input_shape=(WINDOW_SIZE, X_win.shape[2])),
    Dense(1)
])
model.compile(optimizer=Adam(LEARNING_RATE), loss='mse')

history = model.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=0.1,
    verbose=2
)

# 4) Predicting
y_pred_scaled = model.predict(X_test)
y_pred = target_scaler.inverse_transform(y_pred_scaled).flatten()
y_true = target_scaler.inverse_transform(y_test).flatten()

# 4.1) Accuracy metrics
mse = mean_squared_error(y_true, y_pred)
mae = mean_absolute_error(y_true, y_pred)
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# 4.2) Directional
actual_series = df[TARGET_COL].values
test_start_idx = len(actual_series) - PRED_DAYS
prev_actual = actual_series[test_start_idx - 1]
actual_test = actual_series[test_start_idx:]
# 4.3) Directions calculation
pred_dirs = np.where(y_pred > np.concatenate([[prev_actual], actual_test[:-1]]), 1, -1)
true_dirs = np.where(actual_test > np.concatenate([[prev_actual], actual_test[:-1]]), 1, -1)
directional_acc = (pred_dirs == true_dirs).mean() * 100

# 4.4) Print
print(f"MSE on last {PRED_DAYS} days: {mse:.4f}")
print(f"MAE on last {PRED_DAYS} days: {mae:.4f}")
print(f"MAPE on last {PRED_DAYS} days: {mape:.2f}%")
print(f"Directional Accuracy: {directional_acc:.2f}%")

# 5) Plotting Actual vs Predicted Last 30 Days
plt.figure(figsize=(8, 6))
plt.plot(test_dates, y_true, label='Actual')
plt.plot(test_dates, y_pred, label='Predicted')

plt.title(f'Actual vs Predicted Closing Prices (Last {PRED_DAYS} Days)')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
